<a href="https://colab.research.google.com/github/timosachsenberg/EuBIC2026/blob/main/notebooks/EUBIC_Task3_Quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (for Google Colab)
!pip install -q pyopenms>=3.5.0 pyopenms-viz>=1.0.0

# Notebook 3 – Quantification

In this tutorial, we demonstrate a complete feature detection and annotation workflow using Biosaur2, an isotope-aware feature detection algorithm, as implemented in pyOpenMS.

In this notebook we will:

1. **Apply the Biosaur2 algorithm to detect isotope-resolved features from mzML data.**

2. **Feature map annotations with peptide identifications.**

3. **Visually inspect detected features in retention time–m/z–intensity space.**



In [ ]:

%matplotlib inline
import os
import pyopenms as oms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print("pyOpenMS version:", oms.__version__)


pyOpenMS version: 3.5.0


In [ ]:
# Download mzML and idXML files from course repository
import os

if not os.path.exists("BSA1.mzML"):
    !wget -q -O "BSA1.mzML" https://raw.githubusercontent.com/timosachsenberg/EuBIC2026/main/data/BSA1.mzML

if not os.path.exists("BSA1_F1.idXML"):
    !wget -q -O "BSA1_F1.idXML" https://raw.githubusercontent.com/timosachsenberg/EuBIC2026/main/data/BSA1_F1.idXML

In [ ]:
# Load mzML file to MSExperiment
exp = oms.MSExperiment()
oms.MzMLFile().load("BSA1.mzML", exp)


# 1. Apply the Biosaur2 algorithm.

**Aims of this task**

- Biosaur2 is an isotope-aware feature detection method that identifies peptide features by clustering peaks across retention time and evaluating their isotopic patterns and charge state consistency.
- This strategy enables reliable discrimination between true peptide signals and background noise, particularly in complex LC–MS datasets.
- As each detected feature represents an aggregated MS1 signal characterized by a centroid mass-to-charge ratio, a retention time apex, an integrated intensity, and an inferred charge state.

**Implementation**
- The Biosaur2 algorithm (`Biosaur2Algorithm()`) instance initialized and provided with the experimental data as `MSExperiment` loaded from mzML file.
- Feature detection was executed using default algorithm parameters, yielding a FeatureMap object containing all detected features.



In [ ]:
# Initialized Biosaur feature detection algorithm
biosaur = oms.Biosaur2Algorithm()

# Run biosaur
biosaur.setMSData(exp)


In [ ]:
# biosaur output as FeatureMap
features = oms.FeatureMap()
biosaur.run(features)

#oms.FeatureXMLFile().store("BSA1.featureXML", features)

# 2. Feature map annotations with peptide identifications.

**Aims of this task**
- To associate MS1-level quantitative features with sequence-level peptide identifications.
- To integrate peptide and protein identification information into the detected feature map.
- To enable biologically interpretable, peptide-resolved quantitative analysis.

**Implementation**
- Peptide and protein identifications were loaded from an idXML file generated by MS/MS database searching.
- An `IDMapper` instance was initialized to perform feature–identification mapping. see: [https://pyopenms.readthedocs.io/en/latest/user_guide/PSM_to_features.html](https://pyopenms.readthedocs.io/en/latest/user_guide/PSM_to_features.html)
- Peptide identifications were annotated onto detected features based on proximity in RT and m/z space.
- The annotated features were stored within the existing `FeatureMap` structure for downstream analysis.

In [ ]:
# Load identification (.idXML) file extract peptides and protein
peptide_ids = oms.PeptideIdentificationList()
protein_ids = []
oms.IdXMLFile().load("BSA1_F1.idXML", protein_ids, peptide_ids)


In [ ]:
# Configure IDMapper
id_mapper = oms.IDMapper()
params = id_mapper.getParameters()
params.setValue("rt_tolerance", 5.0)  # RT tolerance in seconds
params.setValue("mz_tolerance", 10.0)  # m/z tolerance in ppm
id_mapper.setParameters(params)


In [ ]:
id_mapper.annotate(features, peptide_ids, protein_ids, True, True, exp)


# 3. Visually inspect detected features in retention time–m/z–intensity space.

**Aims of this task**
- To visually evaluate the detected MS1 features in retention time–mass-to-charge–intensity space, enabling qualitative assessment of feature detection performance.

**Implementation**
- The detected feature map was converted into a tabular pandas DataFrame for exploratory analysis. see: [https://pyopenms.readthedocs.io/en/latest/user_guide/export_pandas_dataframe.html](https://pyopenms.readthedocs.io/en/latest/user_guide/export_pandas_dataframe.html)
- The plotting backend was configured to enable mass spectrometry–specific visualizations. see: [https://pyopenms-viz.readthedocs.io/en/latest/](https://pyopenms-viz.readthedocs.io/en/latest/)
- A peak map visualization was generated, projecting features in retention time, m/z, and intensity space.


In [ ]:
# Export features into dataframe
df = features.get_df()
df.head(2)

,peptide_sequence,peptide_score,ID_filename,ID_native_id,charge,rt,mz,rt_start,rt_end,mz_start,mz_end,quality,intensity
feature_id,,,,,,,,,,,,,
12592275275208522725,None,NaN,unknown,None,5,2272.696289,674.534031,2252.129150,2299.366455,674.532496,676.139162,9.0,83422.359375
17439098422051461769,None,NaN,None,None,3,2208.773438,655.301653,2195.811768,2217.431396,655.297018,657.308736,7.0,230824.500000


In [ ]:
# interactive PeakMap plot with plotly
from pyopenms_viz._plotly import PLOTLYPeakMapPlot

plot = PLOTLYPeakMapPlot(
    data=df,
    x="rt",
    y="mz",
    z="intensity",
    width=800,
    height=800,
    grid=False,
    add_marginals=True, # showing RT and intensities
)

plot.show();

In [ ]:
# ploting peakmap and having bounding boxes at the feature position
plot = PLOTLYPeakMapPlot(
    data=df,
    x="rt",
    y="mz",
    z="intensity",
    width=1000,
    height=1000,
    grid=False,
)

# Create rectangles for all features
shapes = []
for _, row in df.iterrows():
    shapes.append(
        dict(
            type="rect",
            x0=row["rt_start"],
            x1=row["rt_end"],
            y0=row["mz_start"],
            y1=row["mz_end"],
            line=dict(color="blue", width=1)
        )
    )

# Add all rectangles to the plot
plot.fig.update_layout(shapes=shapes)

# Show the interactive plot
plot.show();

In [ ]:
# Filter features within the RT window
df_cut = df[(df["rt_start"] >= 1600) & (df["rt_end"] <= 1650)]

# Plot peakmap
plot = PLOTLYPeakMapPlot(
    data=df_cut,
    x="rt",
    y="mz",
    z="intensity",
    width=1000,
    height=1000,
    grid=False,
)

# Create rectangles for filtered features
shapes = []
for _, row in df_cut.iterrows():
    shapes.append(
        dict(
            type="rect",
            x0=row["rt_start"],
            x1=row["rt_end"],
            y0=row["mz_start"],
            y1=row["mz_end"],
            line=dict(color="blue", width=1)
        )
    )

# Add rectangles to the plot
plot.fig.update_layout(shapes=shapes)

# Show the interactive plot
plot.show();